In [121]:
# Excel 文件路径，设置月份，分公司
file_path = '/Users/litianyu/Downloads/pest/ruixing/Aoken service 2024 data.xlsx'
target_month = 7  # 目标月份为 7 月
target_location = '福建'  # 目标分公司为 '福建'

In [122]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
import docx

In [123]:
# 每个 sheet 对应的日期列名
date_columns = {
    '城市得分': '归属月份',
    '按期消杀&门店评价': '执行日期',
    '消杀稽核': '检查完成日期',
    '配合度': '评价归属月份',
    '异常管理': '执行日期',
    '虫害客诉': '反馈日期' 
}
branch_column = '分公司'  # 分公司列名在各 sheet 中都改为 '分公司'

# 存储每个 sheet 中 7 月份且分公司为 '福建' 的数据
monthly_data_filtered = {}

# 读取整个 Excel 文件中的所有 sheet
excel_data = pd.ExcelFile(file_path)

# 遍历每个 sheet
for sheet_name, date_col in date_columns.items():
    # 读取特定 sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # 将日期列转换为日期格式
    df[date_col] = pd.to_datetime(df[date_col], format='%m/%d/%y', errors='coerce')
    
    # 筛选出 7 月份的数据，并筛选出 '分公司' 列为 '福建' 的行
    month_data = df[(df[date_col].dt.month == target_month) & (df[branch_column] == target_location)]
    
    # 存储筛选后的数据
    monthly_data_filtered[sheet_name] = month_data
    
'''
# 查看每个 sheet 中 7 月份且分公司为 '福建' 的数据
for sheet, data in monthly_data_filtered.items():
    print(f"数据来自 sheet: {sheet}")
    print(data)
    print("\n" + "-"*50 + "\n")
'''

'\n# 查看每个 sheet 中 7 月份且分公司为 \'福建\' 的数据\nfor sheet, data in monthly_data_filtered.items():\n    print(f"数据来自 sheet: {sheet}")\n    print(data)\n    print("\n" + "-"*50 + "\n")\n'

# Part 01 整体服务情况

## 月度整体服务情况

In [124]:
# 从筛选后的数据中读取 ’城市得分‘ 的数据
data_score = monthly_data_filtered['城市得分']

# 计算两个列的和
sum1 = data_score['消杀店数（自营）'].sum()  # 自营列
sum2 = data_score['消杀店数（联营）'].sum()  # 联营列
sum3 = sum1 + sum2  # 总和
# 统计城市数量
count_cities = data_score['城市'].nunique()
# 计算服务得分
avg_score = data_score['月度得分'].mean()

# 按指定格式打印结果
print(f"消杀门店{sum3}家（其中自营{sum1}家，联营{sum2}家）")
print(f"服务城市数量{count_cities}个")
print(f"整体服务得分：{avg_score:.1f}") #保留一位小数

消杀门店794家（其中自营537家，联营257家）
服务城市数量9个
整体服务得分：93.7


In [125]:
TBU_report_year=datetime.now().year
TBU_report_month=target_month
TBU_province=target_location
TBU_service_month=target_month
TBU_no_store=sum3
TBU_indep_store=sum1
TBU_union_store=sum2
TBU_no_city=count_cities
TBU_service_score=avg_score
TBU_nation_score=94.95

In [126]:
TBU_province

'福建'

## 未按期清单

In [127]:
# ! pip install -U kaleido

In [128]:
# 从筛选后的数据中读取 ’按期消杀&门店评价‘ 的数据
data_disinfection = monthly_data_filtered['按期消杀&门店评价']

data_not_disinfect = data_disinfection[data_disinfection['按期消杀'] == 0]
data_selected = data_not_disinfect[['门店名称','消杀类型','计划日期','执行日期','执行人']].copy()
data_selected['差异天数'] = (data_selected['执行日期'] - data_selected['计划日期']).dt.days

# Ensure columns are in datetime format
data_selected['计划日期'] = pd.to_datetime(data_selected['计划日期'], errors='coerce')
data_selected['执行日期'] = pd.to_datetime(data_selected['执行日期'], errors='coerce')

# Format the date columns to 'yyyy-mm-dd'
data_selected['计划日期'] = data_selected['计划日期'].dt.strftime('%Y-%m-%d')
data_selected['执行日期'] = data_selected['执行日期'].dt.strftime('%Y-%m-%d')

# 转换 DataFrame 为 Markdown 表格
markdown_table = data_selected.to_markdown(index=False)

if not data_selected.empty:
    # 打印 Markdown 格式的表格，Word 会自动处理列宽对齐
    print(markdown_table)
else:
    print("本月没有未按期清单")

| 门店名称         | 消杀类型   | 计划日期   | 执行日期   | 执行人   |   差异天数 |
|:-----------------|:-----------|:-----------|:-----------|:---------|-----------:|
| 白马南路店       | 常规消杀   | 2024-07-22 | 2024-07-23 | 韦昌莹   |          1 |
| 福州中央第五街店 | 常规消杀   | 2024-07-14 | 2024-07-15 | 潘登铿   |          1 |
| 紫阳国际大厦店   | 紧急消杀   | 2024-07-09 | 2024-07-12 | 韦昌莹   |          3 |
| 马尾正荣御品店   | 常规消杀   | 2024-07-12 | 2024-07-13 | 韦昌莹   |          1 |


## 各区域服务得分

In [10]:
# 示例数据
data_citiscores = data_score[['城市','月度得分']].copy()

# 创建条形图
fig = px.bar(data_citiscores, 
             x='城市', 
             y='月度得分', 
             title='各区域月度得分', 
             color='城市', 
             labels={'月度得分'}, 
             text=data_citiscores['月度得分'].apply(lambda x: f'{x:.2f}'))  # 格式化为两位小数

# 设置中文字体
fig.update_layout(
    font=dict(family="SimHei"),  # 设置字体为 SimHei，适合显示中文
    title_font=dict(family="SimHei", size=16),  # 设置标题字体
    # xaxis_title_font=dict(family="SimHei", size=14),  # 设置X轴标题字体
    # yaxis_title_font=dict(family="SimHei", size=14),  # 设置Y轴标题字体
    xaxis_title="",  # 隐藏 x 轴标题
    yaxis_title="",  # 隐藏 y 轴标题
    bargap=0.5  # 调整柱状图之间的间隔，值越小柱子越窄
)

# 设置条形图上数字的显示方向为水平
fig.update_traces(textangle=0)  # 设置文本角度为0度（水平显示）
fig.update_traces(hovertemplate='<b>城市</b>: %{x}<br><b>得分</b>: %{y:.2f}')  # 不显示 'text' 数据

# 输出图表
fig.show()


fig.write_image("各区域月度得分.jpg", scale=2)  # Save as jpg

## 虫害客诉情况

In [11]:
# 从筛选后的数据中读取 ’虫害客诉‘ 的数据
data_complaints = monthly_data_filtered['虫害客诉']

# 示例数据
data_complaints_nums = data_complaints[['城市','三级业务类型']].copy()

# 按城市和虫害类型分组，计算每个组合的数量
grouped_data = data_complaints_nums.groupby(['城市', '三级业务类型']).size().reset_index(name='数量')

# 自定义虫害类型顺序，使得 "其他虫害" 排在最后
category_order = ['苍蝇', '飞虫', '蚊子', '蟑螂', '其他虫害']
available_categories = grouped_data['三级业务类型'].unique()
category_order = [category for category in category_order if category in available_categories] # modify the category_order list to include only categories that are actually present in the data
grouped_data['三级业务类型'] = pd.Categorical(grouped_data['三级业务类型'], categories=category_order, ordered=True)

# 计算每个城市的总投诉数量
city_totals = grouped_data.groupby('城市')['数量'].sum().reset_index()
city_totals.columns = ['城市', '总投诉数量']
# 按总投诉数量升序排序城市
city_totals = city_totals.sort_values(by='总投诉数量', ascending=True)

# 图1：显示每个城市的虫害类型总投诉数量
fig1 = px.bar(city_totals,
              x='城市',
              y='总投诉数量',
              title="各区域虫害客诉",
              color='城市',
              text=city_totals['总投诉数量'])

# 设置中文字体和布局
fig1.update_layout(
    font=dict(family="SimHei"),
    title_font=dict(family="SimHei", size=16),
    xaxis_title="",  # 隐藏 x 轴标题
    yaxis_title="",
)
fig1.update_traces(textangle=0)  # 设置文本角度为0度（水平显示）

fig1.show()

fig1.write_image("各区域虫害客诉.jpg", scale=2)  # Save as jpg

In [12]:
# 图2：显示每个城市的不同虫害类型的投诉数量
fig2 = px.bar(grouped_data,
              x='城市',
              y='数量',
              color='三级业务类型',
              title="各区域不同类型虫害客诉",
              labels={'三级业务类型': '虫害类型'},
              category_orders={'三级业务类型': category_order},  # 使用自定义的虫害类型顺序
              text=grouped_data['数量'])

# 设置中文字体和布局
fig2.update_layout(
    font=dict(family="SimHei"),
    title_font=dict(family="SimHei", size=16),
    xaxis_title="",  # 隐藏 x 轴标题
    yaxis_title="",
    barmode='group',  # 分组显示每个城市的虫害种类
)
fig2.update_traces(textangle=0)  # 设置文本角度为0度（水平显示）

# 显示图表
fig2.show()

fig2.write_image("各区域不同类型虫害客诉.jpg", scale=2)  # Save as jpg

# Part 02 紧急服务分析

## 虫害客诉趋势分析

In [13]:
# 虫害客诉sheet，所有月份的data
all_months_data_complaints = pd.read_excel(file_path, sheet_name='虫害客诉')
# 摘取其中target_location（分公司）的数据
all_months_target_location = all_months_data_complaints[all_months_data_complaints[branch_column] == target_location].copy()

# 将 '反馈日期' 列转换为日期格式
all_months_target_location['反馈日期'] = pd.to_datetime(all_months_target_location['反馈日期'], errors='coerce')

# 从日期中提取月份信息
all_months_target_location.loc[:, '月份'] = all_months_target_location['反馈日期'].dt.month

# 按月份分组并统计虫害客诉数量
monthly_complaints = all_months_target_location.groupby('月份').size().reset_index(name='客诉数量')

# 过滤数据，只保留到目标月份的数据
target_monthly_complaints = monthly_complaints[monthly_complaints['月份'] <= target_month]

# 检查是否生成正确的数据
# print(target_monthly_complaints)

# 绘制月度趋势图
fig = px.line(target_monthly_complaints, x='月份', y='客诉数量', title="虫害客诉趋势", text='客诉数量',
              labels={'月份': '月份', '客诉数量': '客诉数量'})

# 设置中文字体
fig.update_layout(
    font=dict(family="SimHei"),
    title_font=dict(family="SimHei", size=16),
    xaxis_title="月份",
    yaxis_title="",
)

# 设置文本标签的位置和样式
fig.update_traces(
    textposition="bottom center",  # 文本标签显示在数据点下方
    textfont=dict(size=12, color="black")  # 调整文本字体大小和颜色
)

# 设置x轴只显示整月
fig.update_xaxes(
    tickmode="array",  # 使用指定的数组作为刻度
    tickvals=target_monthly_complaints['月份'],  # 指定刻度值为数据中的月份
    ticktext=[f"{int(month)}月" for month in target_monthly_complaints['月份']]  # 用中文标记月份
)
# 显示图表
fig.show()

fig.write_image("虫害客诉趋势.jpg", scale=2)  # Save as jpg

## 自营 VS 联营
## 自取 VS 外卖 

In [14]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

# 获取门店运营模式的所有可能值
yunying_categories = ['联营', '自营']  # 明确列出所有门店运营模式的类别

# 计算每个组合的数量
grouped_yunying_data = data_complaints.groupby(['门店运营模式']).size().reset_index(name='数量')

# Create DataFrame for all possible categories
yunying_data_all = pd.DataFrame(yunying_categories, columns=['门店运营模式'])

# Merge with actual data, filling missing categories with 0
yunying_data = pd.merge(yunying_data_all, grouped_yunying_data, how='left', on='门店运营模式').fillna(0)

# 图1：门店运营模式对比
fig1 = px.bar(yunying_data,
              x='门店运营模式',
              y='数量',
              title="门店运营模式对比",
              color='门店运营模式',
              text=yunying_data['数量'])

# 设置中文字体和布局
fig1.update_layout(
    font=dict(family="SimHei"),
    title_font=dict(family="SimHei", size=16),
    xaxis_title="",  # 隐藏 x 轴标题
    yaxis_title="",
)
fig1.update_traces(textangle=0, showlegend=False)  # 设置文本角度为0度（水平显示）

# 订单类型的处理
dingdan_categories = data_complaints['订单类型'].unique()  # 获取所有订单类型
grouped_dingdan_data = data_complaints.groupby(['订单类型']).size().reset_index(name='数量')
dingdan_data_all = pd.DataFrame(dingdan_categories, columns=['订单类型'])
dingdan_data = pd.merge(dingdan_data_all, grouped_dingdan_data, how='left', on='订单类型').fillna(0)

# 图2：订单类型对比
fig2 = px.bar(dingdan_data,
              x='订单类型',
              y='数量',
              title="订单类型对比",
              color='订单类型',
              text=dingdan_data['数量'])

# 设置中文字体和布局
fig2.update_layout(
    font=dict(family="SimHei"),
    title_font=dict(family="SimHei", size=16),
    xaxis_title="",  # 隐藏 x 轴标题
    yaxis_title="",
    barmode='group',  # 分组显示每个城市的虫害种类
)
fig2.update_traces(textangle=0, showlegend=False)  # 设置文本角度为0度（水平显示）

# 创建子图
fig = make_subplots(
    rows=1, cols=2,  # 一行两列
    subplot_titles=("门店运营模式对比", "订单类型对比"),  # 子图标题
    column_widths=[0.4, 0.4],
    horizontal_spacing=0.25  # 增加子图间距（默认是 0.02）
)

# Add all traces from fig1, fig2 to the subplot
for trace in fig1['data']:
    fig.add_trace(trace, row=1, col=1)
    
for trace in fig2['data']:
    fig.add_trace(trace, row=1, col=2)

# Update layout settings
fig.update_layout(
    title_text="",
    font=dict(family="SimHei"),
)

# Show the combined figure
fig.show()

fig.write_image("门店运营模式对比 & 订单类型对比.jpg", scale=2)  # Save as jpg

## 虫害客诉重点门店（虫害客诉≥3单的门店）

In [146]:
# 分组，计算每个组合的数量
grouped_mendian_data = data_complaints.groupby(['门店','城市']).size().reset_index(name='数量')
# 示例数据
grouped_mendian_data = grouped_mendian_data[['门店','城市','数量']].copy() 
# 计算数量
mendian_data = grouped_mendian_data.groupby(['门店','城市'])['数量'].sum().reset_index()
# 过滤数据，只保留大于三单的数据
target_mendian_data = mendian_data[mendian_data['数量'] >= 3]# 图3：门店数量的表格展示
fig_table = go.Figure(data=[
    go.Table(
        header=dict(values=["门店", "城市", "数量"], font=dict(size=12, color="black"), align="center"),
        cells=dict(values=[target_mendian_data['门店'], target_mendian_data['城市'], target_mendian_data['数量']], align="center")
    )
])

# # 设置更宽的尺寸单独展示表格
# fig_table.update_layout(
#     title='重点门店'
# )

# if not target_mendian_data.empty:
#     fig_table.show()
# else:
#     print("本月所有门店的虫害客诉均小于三单")
    

# fig_table.write_image("重点门店.jpg", scale=2)  # Save as jpg

# Part 03 虫害分布和风险？ 消杀建议

In [16]:
# 从筛选后的数据中读取 ’异常管理‘ 的数据
data_abnormal = monthly_data_filtered['异常管理']

# 示例数据
data_abnormal = data_abnormal[['门店名称','所属城市','消杀建议']].copy()

# Function to add <br> tags every 20 characters
def add_html_breaks(text, line_length=20):
    return '<br>'.join(text[i:i + line_length] for i in range(0, len(text), line_length))

# Apply line breaks to '消杀建议' column
data_abnormal['消杀建议'] = [add_html_breaks(text) for text in data_abnormal['消杀建议']]

# Filter rows where '消杀建议' is not empty
suggestion_data = data_abnormal[(data_abnormal['消杀建议'].notna())]

# Check if the filtered data has any rows
if not suggestion_data.empty:
    # Extract the relevant columns for the table
    suggestion_data = suggestion_data[['门店名称','所属城市','消杀建议']]

    table_trace = go.Table(
        header=dict(
            values=["门店名称", "城市", "消杀建议"],
            font=dict(size=10, color="black"),
            align="center"),
        cells=dict(
            values=[suggestion_data['门店名称'], suggestion_data['所属城市'], suggestion_data['消杀建议']],
            align="center")
    ) 

    # Display the table as a figure
    fig = go.Figure(data=[table_trace])

    # Update layout settings for a clean look
    fig.update_layout(
    title_text="消杀建议",
    font=dict(family="SimHei"),
    autosize=False,
    width=1000,  # Increase the width
    height=800   # Increase the height
    )

    # Show the table
    fig.show()
else:
    print("所有门店均无异常")
    
fig.write_image("消杀建议.jpg", width=1000, height=800, scale=2)  # Save as jpg



# Part 04 门店问题汇总

## 灭蝇灯

In [17]:
# 示例数据
data_mieyingdeng = data_disinfection[['门店名称','灭蝇灯']].copy()

# Filter rows where the '灭蝇灯' column does not match the specified standard text
filtered_data = data_mieyingdeng[data_mieyingdeng['灭蝇灯'] != '符合标准：所有灯管均正常，位置&数量符合虫害防控要求，灭蝇灯无缺失']

# Check if the filtered data has any rows
if not filtered_data.empty:
    # Extract the relevant columns for the table: '门店名称' and '灭蝇灯'
    table_data = filtered_data[['门店名称', '灭蝇灯']]
    
    # Create the Plotly table with updated label
    table_trace = go.Table(
        header=dict(values=["门店名称", "具体描述"], font=dict(size=12, color="black"), align="center"),
        cells=dict(values=[table_data['门店名称'], table_data['灭蝇灯']], align="center")
    )

    # Display the table as a figure
    fig = go.Figure(data=[table_trace])

    # Update layout settings for a clean look
    fig.update_layout(
        title_text="灭蝇灯",
        font=dict(family="SimHei"),
    )

    # Show the table
    fig.show()
else:
    print("所有门店的灭蝇灯均符合标准")
    
fig.write_image("灭蝇灯.jpg",  scale=2)  # Save as jpg

所有门店的灭蝇灯均符合标准


## 风幕机

In [18]:
# 示例数据
data_fengmuji = data_disinfection[['门店名称','风幕机']].copy()

# Filter rows where the '灭蝇灯' column does not match the specified standard text
filtered2_data = data_fengmuji[data_fengmuji['风幕机'] != '符合标准：覆盖完整，风力及地、符合要求']

# Check if the filtered data has any rows
if not filtered2_data.empty:
    # Extract the relevant columns for the table: '门店名称' and '灭蝇灯'
    table2_data = filtered2_data[['门店名称', '风幕机']]
    
    # Create the Plotly table with updated label
    table_trace = go.Table(
        header=dict(values=["门店名称", "具体描述"], font=dict(size=12, color="black"), align="center"),
        cells=dict(values=[table2_data['门店名称'], table2_data['风幕机']], align="center")
    )

    # Display the table as a figure
    fig = go.Figure(data=[table_trace])

    # Update layout settings for a clean look
    fig.update_layout(
        title_text="风幕机",
        font=dict(family="SimHei"),
    )

    # Show the table
    fig.show()
else:
    print("所有门店的风幕机均符合标准")
    
    
fig.write_image("风幕机.jpg", width=1000, height=800, scale=2)  # Save as jpg

## 不满意原因

In [147]:
# 示例数据
data_reason = data_disinfection[['门店名称','不满意原因']].copy()

# Filter rows where '不满意原因' is neither empty nor "其他"
filtered3_data = data_reason[(data_reason['不满意原因'].notna()) & (~data_reason['不满意原因'].isin(['', '其他']))]


# Check if the filtered data has any rows
if not filtered3_data.empty:
    # Extract the relevant columns for the table: '门店名称' and '灭蝇灯'
    table3_data = filtered3_data[['门店名称', '不满意原因']]
    
    # Create the Plotly table with updated label
    table_trace1 = go.Table(
        header=dict(values=["门店名称", "具体描述"], font=dict(size=12, color="black"), align="center"),
        cells=dict(values=[table3_data['门店名称'], table3_data['不满意原因']], align="center")
    )

    # Display the table as a figure
    fig = go.Figure(data=[table_trace1])

    # Update layout settings for a clean look
    fig.update_layout(
        title_text="不满意原因",
        font=dict(family="SimHei"),
    )

    # Show the table
    fig.show()
else:
    print("本月所有门店对服务均满意")
    

    
fig.write_image("不满意原因.jpg", width=1000, height=800, scale=2)  # Save as jpg


# Update the monthly report

In [163]:
import pandas as pd
from docx import Document
from docx.oxml import OxmlElement

def insert_table_to_paragraph(doc, df, paragraph_name, title="Table"):
    """
    Inserts a table into the specified paragraph of a Word document.

    Parameters:
        doc (Document): The Word document object
        df (DataFrame): The pandas DataFrame to insert as a table
        paragraph_name (str): The name of the paragraph under which to insert the table
        title (str): The title to display above the table
    """
    # Convert the DataFrame to a list of lists (structured format for table)
    table_data = [df.columns.tolist()] + df.values.tolist()

    # Add the table under the specified paragraph
    for i, paragraph in enumerate(doc.paragraphs):
        if paragraph_name in paragraph.text:
            # Add a title for the table
            title_paragraph = doc.add_paragraph(title)  # Title text
            title_paragraph.alignment = 1  # Center alignment for the title
            
            # Add the table below the paragraph
            table = doc.add_table(rows=len(table_data), cols=len(table_data[0]))

            # Apply 'Table Grid' style if available or fall back to 'Table Normal'
            try:
                table.style = 'Table Grid'  # Apply the 'Table Grid' style with borders
            except KeyError:
                table.style = 'Table Normal'  # Apply a basic table style if 'Table Grid' isn't available

            # Populate the header row
            for j, header in enumerate(table_data[0]):
                table.cell(0, j).text = header

            # Populate the data rows
            for row_idx, row_data in enumerate(table_data[1:], start=1):
                row_cells = table.rows[row_idx].cells
                for col_idx, cell_data in enumerate(row_data):
                    row_cells[col_idx].text = str(cell_data)

            # Apply borders (grid lines) to each cell using python-docx' table styles
            for row in table.rows:
                for cell in row.cells:
                    # Center the text horizontally
                    for paragraph in cell.paragraphs:
                        paragraph.alignment = 1  # Horizontal center alignment

            # Insert the title and table directly below the paragraph
            # Add the title and table after the paragraph without using addnext
            doc.paragraphs[i]._element.addnext(title_paragraph._element)
            doc.paragraphs[i]._element.addnext(table._element)

            break  # Exit loop after placing the table and title


In [164]:
doc = Document("/Users/litianyu/Downloads/pest/ruixing/虫害分析月度报告 -- 样本2.docx")

# Call the function
insert_table_to_paragraph(doc, data_selected, "未按期清单", "Table 1: 未按期清单")

# Save the changes to the document
doc.save('/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx')

In [165]:

doc = Document("/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx")

# Call the function
insert_table_to_paragraph(doc, target_mendian_data, "重点门店", "Table 2: 重点门店")

# Save the changes to the document
doc.save('/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx')

In [166]:

doc = Document("/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx")

# Call the function
insert_table_to_paragraph(doc, suggestion_data, "消杀建议", "Table 3: 消杀建议")

# Save the changes to the document
doc.save('/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx')

In [167]:

doc = Document("/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx")

# Call the function
insert_table_to_paragraph(doc, table2_data, "风幕机", "Table 4: 风幕机")

# Save the changes to the document
doc.save('/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx')

In [168]:

doc = Document("/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx")

# Call the function
insert_table_to_paragraph(doc, table3_data, "不满意原因", "Table 5: 不满意原因")

# Save the changes to the document
doc.save('/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx')

In [101]:
÷


In [87]:
styles = [style.name for style in doc.styles]
print(styles)

['normal', 'Table Normal', 'Heading 1', 'Heading 2', 'Heading 3', 'Heading 4', 'Heading 5', 'Heading 6', 'Title', 'Subtitle']


In [169]:
from docx import Document

# Function to replace text
def replace_text_in_paragraph(paragraph, replacements):
    for old_text, new_text in replacements.items():
        if old_text in paragraph.text:
            paragraph.text = paragraph.text.replace(old_text, new_text)

def replace_text_in_word(input_path, output_path, replacements):
    # Load the Word document
    doc = Document(input_path)
    
    # Iterate through all paragraphs
    for paragraph in doc.paragraphs:
        replace_text_in_paragraph(paragraph, replacements)
    
    # Iterate through tables (if the document has tables)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    replace_text_in_paragraph(paragraph, replacements)
    
    # Save the updated document
    doc.save(output_path)

# Define replacements
replacements = {
    '#TBU_report_year#': str(TBU_report_year),
    '#TBU_report_month#': str(TBU_report_month),
    '#TBU_province#': TBU_province,
    '#TBU_service_month#': str(TBU_service_month),
    '#TBU_no_store#': str(TBU_no_store),
    '#TBU_indep_store#': str(TBU_indep_store),
    '#TBU_union_store#': str(TBU_union_store),
    '#TBU_no_city#': str(TBU_no_city),
    '#TBU_service_score#': f"{TBU_service_score:.2f}",
    '#TBU_nation_score#': "94.95"
}

# Call the function
replace_text_in_word(
    "/Users/litianyu/Downloads/pest/ruixing/modified_虫害分析月度报告-table.docx",
    "/Users/litianyu/Downloads/pest/ruixing/虫害分析月度报告-text.docx",
    replacements)

In [170]:
from docx import Document
from docx.shared import Inches

# Function to insert an image into a Word document
def insert_graph(doc, placeholder, image_path, width, height):
    """
    Inserts an image at the location of a placeholder in a Word document.

    Parameters:
        doc (Document): The Word document object.
        placeholder (str): The placeholder text to replace with the image.
        image_path (str): The path to the image file.
        width (float): The width of the image (in inches).
        height (float): The height of the image (in inches).
    """
    for paragraph in doc.paragraphs:
        if placeholder in paragraph.text:
            # Replace placeholder text with the image
            paragraph.text = paragraph.text.replace(placeholder, "")
            run = paragraph.add_run()
            run.add_picture(image_path, width=Inches(width), height=Inches(height))
            break

# Load the Word document
input_path = "/Users/litianyu/Downloads/pest/ruixing/虫害分析月度报告-text.docx"
output_path = "/Users/litianyu/Downloads/pest/ruixing/虫害分析月度报告-plot.docx"
doc = Document(input_path)

# Insert images at designated placeholders
insert_graph(doc, "未按期清单", "按期消杀门店评价.jpg", 5, 3)
insert_graph(doc, "各区域服务得分", "各区域月度得分.jpg", 5, 3)
insert_graph(doc, "各区域虫害客诉", "各区域虫害客诉.jpg", 5, 3)
insert_graph(doc, "各区域不同类型虫害客诉", "各区域不同类型虫害客诉.jpg", 5, 3)
insert_graph(doc, "虫害客诉趋势", "虫害客诉趋势.jpg", 5, 3)
insert_graph(doc, "门店运营模式对比 & 订单类型对比", "门店运营模式对比 & 订单类型对比.jpg", 5, 3)
insert_graph(doc, "重点门店", "重点门店.jpg", 5, 3)
insert_graph(doc, "消杀建议", "消杀建议.jpg", 5, 3)
insert_graph(doc, "风幕机", "风幕机.jpg", 5, 3)
insert_graph(doc, "不满意原因", "不满意原因.jpg", 5, 3)

# Save the updated document
doc.save(output_path)


In [23]:
# Update the text

# for paragraph in doc.paragraphs:
#     replace_text(paragraph, '#TBU_report_year#', TBU_report_year)
#     replace_text(paragraph, '#TBU_report_month#', TBU_report_month)
#     replace_text(paragraph, '#TBU_province#', TBU_province)
#     replace_text(paragraph, '#TBU_service_month#', TBU_service_month)
#     replace_text(paragraph, '#TBU_no_store#', TBU_no_store)
#     replace_text(paragraph, '#TBU_indep_store#', TBU_indep_store)
#     replace_text(paragraph, '#TBU_union_store#', TBU_union_store)
#     replace_text(paragraph, '#TBU_no_city#', TBU_no_city)
#     replace_text(paragraph, '#TBU_service_score#', TBU_service_score)
#     replace_text(paragraph, '#TBU_nation_score#', TBU_nation_score)

NameError: name 'replace_text' is not defined

In [ ]:
# # Update the graphs and tables

# insert_graph(doc, '未按期清单', '按期消杀门店评价.jpg', 1,5,3)
# insert_graph(doc, '各区域服务得分', '各区域月度得分.jpg', 1,5,3)
# insert_graph(doc, '各区域虫害客诉', '各区域虫害客诉.jpg', 1,5,3)
# insert_graph(doc, '各区域不同类型虫害客诉', '各区域不同类型虫害客诉.jpg', 1,5,3)
# insert_graph(doc, '虫害客诉趋势', '虫害客诉趋势.jpg', 1,5,3)
# insert_graph(doc, '门店运营模式对比 & 订单类型对比', '门店运营模式对比 & 订单类型对比.jpg', 1,5,3)
# insert_graph(doc, '重点门店', '重点门店.jpg', 1,5,3)
# insert_graph(doc, '消杀建议', '消杀建议.jpg', 1,5,3)
# insert_graph(doc, '风幕机', '风幕机.jpg', 1,5,3)
# insert_graph(doc, '不满意原因', '不满意原因.jpg', 1,5,3)

In [ ]:
# # Save the updated document
# doc.save(output_path)
# print(f"Replacements completed. Updated document saved to: {output_path}")

In [ ]:
# # save the updated document

# doc.save('/Users/litianyu/Downloads/pest/ruixing/虫害分析月度报告-new.docx')